## Identification Rate Metric

При обучении модели для распознавания лиц с помощью CE (кросс-энтропии) мы можем считать метрику accuracy как индикатор того, насколько хорошо наша модель работает. Но у accuracy тут есть недостаток: она не сможет померить, насколько хорошо наша модель работает на лицах людей, которых нет в обучающей выборке.  

Чтобы это исправить, придумали новую метрику: **identification rate**. Вот как она работает:

Создадим два набора изображений лиц: query и distractors. Никакие лица из этих наборов не должны содержаться в обучающем и валидационном датасете.

1. Посчитаем косинусные сходства между лицами, соответствующими одним и тем же людям из query части. Например, пусть одному человеку соответствуют три фото в query: 01.jpg, 02.jpg, 03.jpg. Тогда считаем три косинусных сходства между всеми тремя парами из этих фото.
2. Посчитаем косинусные сходства между лицами, соответствующими разным людям из query части.
3. Посчитаем косинусные сходства между всеми парами лиц из query и distractors. Т.е. пара — это (лицо из query, лицо из distractors). Всего получится |query|*|distractors| пар.
4. Сложим количества пар, полученных на 2 и 3 шагах. Это количество false пар.
5. Зафиксируем **FPR** (false positive rate). Пусть, например, будет 0.01. FPR, умноженный на количество false пар из шага 4 — это разрешенное количество false positives, которые мы разрешаем нашей модели. Обозначим это количество через N.
6. Отсортируем все значения косинусных сходств false пар в порядке убывания. N — ое значение расстояния зафиксируем как **пороговое расстояние**.
7. Посчитаем количество positive пар с шага 1, которые имеют косинусное сходство больше, чем пороговое расстояние. Поделим это количество на общее количество positive пар с шага 1. Это будет TPR (true positive rate) — итоговое значение нашей метрики.

Такая метрика обычно обозначается как TPR@FPR=0.01. FPR может быть разным. Приразных FPR будет получаться разное TPR.

Смысл этой метрики в том, что мы фиксируем вероятность ошибки вида false positive, т.е. когда "сеть сказала, что это один и тот же человек, но это не так", считаем порог косинусного сходства для этого значения ошибки, потом берем все positive пары и смотрим, у скольких из них сходство меньше этого порога. Т.е. насколько точно наша сеть ищет похожие лица при заданной вероятности ошибки вида false positive.

**Для подсчета метрик, то вам нужно разбить данные на query и distractors самим.**

Делается это примерно так:
- Выбраете несколько id, которые не использовались при тренировке моделей, и помещаете их в query set;
- Выбираете несколько id, которые не использовались при тренировке моделей и не входят в query, и помещаете их в distractors set. Обычно distractors set должен быть сильно больше, чем query set.
- Обрабатываете картинки из query и distractors тем же способом, что картинки для обучения сети.


Обратите внимание, что если картинок в query и distractors очень много, то полученных пар картинок в пунктах 1-2-3 алгоритма подсчета TPR@FPR будет очень-очень много. Чтобы код подсчета работал быстрее, ограничивайте размеры этих датасетов. Контролируйте, сколько значений расстояний вы считаете.

Ниже дан шаблон кода для реализации FPR@TPR метрики и ячейки с тестами. Тесты проверяют, что ваш код в ячейках написан правильно.

## План заданий

* Правильно разбить датасет на query и distractors
* Реализовать метрику и пройти все тесты
* Подгрузить все модели, обученные на разных лоссах и сравнить их метрики

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import timm
import cv2
import math
import io
import glob
import os

from PIL import Image
from torchvision import transforms
from torchvision.transforms import v2
import torch.nn.functional as F
from itertools import combinations

In [2]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Шаблон кода для Identificaton rate metric (TPR@FPR)

In [3]:
def compute_embeddings(model, images_list):
    '''
    compute embeddings from the trained model for list of images.
    params:
        model: trained nn model that takes images and outputs embeddings
        images_list: list of images paths to compute embeddings for
    output:
        list: list of model embeddings. Each embedding corresponds to images
              names from images_list
    '''
    # Check if GPU is available
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    model.eval()  # Set model to evaluation mode

    # Define image preprocessing transformations
    # Adjust these transforms based on your model's expected input
    transform = transforms.Compose([
        transforms.Resize((256, 256)),  # Resize to common size, adjust as needed
        transforms.ToTensor(),
        transforms.Normalize(mean=[0, 0, 0],  # ImageNet stats
                           std=[1, 1, 1])
    ])

    embeddings_list = []

    with torch.no_grad():  # Disable gradient calculation
        for image_path in images_list:
            try:
                # Load and preprocess image
                image = Image.open(image_path).convert('RGB')
                image_tensor = transform(image).unsqueeze(0)  # Add batch dimension
                image_tensor = image_tensor.to(device)

                # Forward pass through the model
                embedding, _ = model(image_tensor, torch.zeros(image_tensor.shape[0], device=device))

                # Move to CPU and convert to numpy/list
                embedding = embedding.cpu().numpy().flatten().tolist()
                embeddings_list.append(embedding)

            except Exception as e:
                print(f"Error processing image {image_path}: {e}")
                embeddings_list.append(None)  # Or handle differently

    return embeddings_list

In [4]:
def compute_cosine_query_pos(query_dict, query_img_names, query_embeddings):
    '''
    compute cosine similarities between positive pairs from query (stage 1)
    params:
    query_dict: dict {class: [image_name_1, image_name_2, ...]}. Key: class in
                the dataset. Value: images corresponding to that class
    query_img_names: list of images names
    query_embeddings: list of embeddings corresponding to query_img_names
    output:
    list of floats: similarities between embeddings corresponding
                    to the same people from query list
    '''
    # Create mapping from image name to embedding
    name_to_embedding = {name: emb for name, emb in zip(query_img_names, query_embeddings)}

    similarities = []

    # For each class (person)
    for class_name, img_list in query_dict.items():
        # Get embeddings for all images of this class
        class_embeddings = []
        for img_name in img_list:
            if img_name in name_to_embedding:
                class_embeddings.append(name_to_embedding[img_name])

        # Generate all unique pairs of embeddings for this class
        for emb1, emb2 in combinations(class_embeddings, 2):
            # Compute cosine similarity
            cos_sim = np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))
            similarities.append(float(cos_sim))

    return similarities
    raise NotImplementedError

def compute_cosine_query_neg(query_dict, query_img_names, query_embeddings):
    '''
    compute cosine similarities between negative pairs from query (stage 2)
    params:
    query_dict: dict {class: [image_name_1, image_name_2, ...]}. Key: class in
                the dataset. Value: images corresponding to that class
    query_img_names: list of images names
    query_embeddings: list of embeddings corresponding to query_img_names
    output:
    list of floats: similarities between embeddings corresponding
                    to different people from query list
    '''
    # Create mapping from image name to embedding
    name_to_embedding = {name: emb for name, emb in zip(query_img_names, query_embeddings)}

    # Create list of (class_name, embedding) pairs
    class_embeddings_list = []
    for class_name, img_list in query_dict.items():
        for img_name in img_list:
            if img_name in name_to_embedding:
                class_embeddings_list.append((class_name, name_to_embedding[img_name]))

    similarities = []

    # Compare each embedding with embeddings from different classes
    for i, (class1, emb1) in enumerate(class_embeddings_list):
        for j, (class2, emb2) in enumerate(class_embeddings_list[i+1:], i+1):
            if class1 != class2:  # Only consider different classes
                # Compute cosine similarity
                cos_sim = np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))
                similarities.append(float(cos_sim))

    return similarities
    raise NotImplementedError

def compute_cosine_query_distractors(query_embeddings, distractors_embeddings):
    '''
    compute cosine similarities between negative pairs from query and distractors
    (stage 3)
    params:
    query_embeddings: list of embeddings corresponding to query_img_names
    distractors_embeddings: list of embeddings corresponding to distractors_img_names
    output:
    list of floats: similarities between pairs of people (q, d), where q is
                    embedding corresponding to photo from query, d —
                    embedding corresponding to photo from distractors
    '''
    similarities = []

    # Compute cosine similarity between each query embedding and each distractor embedding
    for q_emb in query_embeddings:
        q_norm = np.linalg.norm(q_emb)
        for d_emb in distractors_embeddings:
            d_norm = np.linalg.norm(d_emb)
            cos_sim = np.dot(q_emb, d_emb) / (q_norm * d_norm)
            similarities.append(float(cos_sim))

    return similarities
    raise NotImplementedError

Ячейка ниже проверяет, что код работает верно:

In [5]:
test_query_dict = {
    2876: ['1.jpg', '2.jpg', '3.jpg'],
    5674: ['5.jpg'],
    864:  ['9.jpg', '10.jpg'],
}
test_query_img_names = ['1.jpg', '2.jpg', '3.jpg', '5.jpg', '9.jpg', '10.jpg']
test_query_embeddings = [
                    [1.56, 6.45,  -7.68],
                    [-1.1 , 6.11,  -3.0],
                    [-0.06,-0.98,-1.29],
                    [8.56, 1.45,  1.11],
                    [0.7,  1.1,   -7.56],
                    [0.05, 0.9,   -2.56],
]

test_distractors_img_names = ['11.jpg', '12.jpg', '13.jpg', '14.jpg', '15.jpg']

test_distractors_embeddings = [
                    [0.12, -3.23, -5.55],
                    [-1,   -0.01, 1.22],
                    [0.06, -0.23, 1.34],
                    [-6.6, 1.45,  -1.45],
                    [0.89,  1.98, 1.45],
]

test_cosine_query_pos = compute_cosine_query_pos(test_query_dict, test_query_img_names,
                                            test_query_embeddings)
test_cosine_query_neg = compute_cosine_query_neg(test_query_dict, test_query_img_names,
                                            test_query_embeddings)
test_cosine_query_distractors = compute_cosine_query_distractors(test_query_embeddings,
                                                            test_distractors_embeddings)

In [6]:
true_cosine_query_pos = [0.8678237233650096, 0.21226104378511604,
                         -0.18355866977496182, 0.9787437979250561]
assert np.allclose(sorted(test_cosine_query_pos), sorted(true_cosine_query_pos)), \
      "A mistake in compute_cosine_query_pos function"

true_cosine_query_neg = [0.15963231223161822, 0.8507997093616965, 0.9272761484302097,
                         -0.0643994061127092, 0.5412660901220571, 0.701307100338029,
                         -0.2372575528216902, 0.6941032794522218, 0.549425446066643,
                         -0.011982733001947084, -0.0466679194884999]
assert np.allclose(sorted(test_cosine_query_neg), sorted(true_cosine_query_neg)), \
      "A mistake in compute_cosine_query_neg function"

true_cosine_query_distractors = [0.3371426578637511, -0.6866465610863652, -0.8456563512871669,
                                 0.14530087113136106, 0.11410510307646118, -0.07265097629002357,
                                 -0.24097699660707042,-0.5851992679925766, 0.4295494455718534,
                                 0.37604478596058194, 0.9909483738948858, -0.5881093317868022,
                                 -0.6829712976642919, 0.07546364489032083, -0.9130970963915521,
                                 -0.17463101988684684, -0.5229363015558941, 0.1399896725311533,
                                 -0.9258034013399499, 0.5295114163723346, 0.7811585442749943,
                                 -0.8208760031249596, -0.9905139680301821, 0.14969764653247228,
                                 -0.40749654525418444, 0.648660814944824, -0.7432584300096284,
                                 -0.9839696492435877, 0.2498741082804709, -0.2661183373780491]
assert np.allclose(sorted(test_cosine_query_distractors), sorted(true_cosine_query_distractors)), \
      "A mistake in compute_cosine_query_distractors function"

И, наконец, финальная функция, которая считает IR metric:

In [7]:
def compute_ir(cosine_query_pos, cosine_query_neg, cosine_query_distractors,
               fpr=0.1):
    '''
    compute identification rate using precomputer cosine similarities between pairs
    at given fpr
    params:
    cosine_query_pos: cosine similarities between positive pairs from query
    cosine_query_neg: cosine similarities between negative pairs from query
    cosine_query_distractors: cosine similarities between negative pairs
                              from query and distractors
    fpr: false positive rate at which to compute TPR
    output:
    float: threshold for given fpr
    float: TPR at given FPR
    '''
    all_negatives = np.concatenate([cosine_query_neg, cosine_query_distractors])

    # Находим порог, соответствующий заданному FPR
    # FPR = доля негативных пар, которые будут неправильно приняты как позитивные
    # Порог выбирается так, чтобы (1 - fpr) негативных пар были ниже порога
    threshold = np.percentile(all_negatives, (1 - fpr) * 100)

    # Вычисляем TPR (True Positive Rate) - долю позитивных пар выше порога
    tpr = np.mean(cosine_query_pos >= threshold)

    return threshold, tpr

И ячейки для ее проверки:

In [8]:
test_thr = []
test_tpr = []
for fpr in [0.5, 0.3, 0.1]:
  x, y = compute_ir(test_cosine_query_pos, test_cosine_query_neg,
                    test_cosine_query_distractors, fpr=fpr)
  test_thr.append(x)
  test_tpr.append(y)

In [9]:
true_thr = [-0.011982733001947084, 0.3371426578637511, 0.701307100338029]
assert np.allclose(np.array(test_thr), np.array(true_thr)), "A mistake in computing threshold"

true_tpr = [0.75, 0.5, 0.5]
assert np.allclose(np.array(test_tpr), np.array(true_tpr)), "A mistake in computing tpr"

А в ячейке ниже вы можете посчитать TPR@FPR для датасета с лицами. Давайте, например, посчитаем для значений fpr = [0.5, 0.2, 0.1, 0.05].

In [10]:
# YOUR CODE HERE

## Загрузка датасета и модели:

In [11]:
# Загрузка тестового датасета
! pip install wldhx.yadisk-direct
! curl -L $(yadisk-direct https://disk.yandex.ru/d/wA_h8o-AV2EFsA) -o test_dataset_aligned.zip
! unzip -qq test_dataset_aligned.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100 9323k  100 9323k    0     0  1336k      0  0:00:06  0:00:06 --:--:-- 1953k


In [12]:
df_test = pd.read_csv('./test_dataset_aligned/test_dataset.csv', index_col=0)
df_test.head()

,image_id,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,...,lefteye_y,righteye_x,righteye_y,nose_x,nose_y,leftmouth_x,leftmouth_y,rightmouth_x,rightmouth_y,set_type
0,020009.jpg,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,158,180,147,156,198,118,223,184,211,query
1,160429.jpg,-1,-1,-1,1,-1,-1,-1,1,-1,...,244,360,241,340,316,253,368,357,360,query
2,030593.jpg,-1,-1,-1,-1,-1,-1,-1,1,-1,...,239,246,228,201,306,148,343,243,329,query
3,123543.jpg,-1,-1,-1,-1,-1,1,1,-1,1,...,215,689,208,646,270,608,317,686,314,query
4,025408.jpg,-1,1,1,1,-1,-1,1,-1,1,...,360,558,347,517,449,449,494,565,468,query


In [13]:
# Загрузка кода модели
! pip install wldhx.yadisk-direct
! curl -L $(yadisk-direct https://disk.yandex.ru/d/0mPiA93X-h5l0A) -o recognizer.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  5224  100  5224    0     0   1877      0  0:00:02  0:00:02 --:--:--  4097


In [14]:
# Загрузка весов модели
! pip install wldhx.yadisk-direct
! curl -L $(yadisk-direct https://disk.yandex.ru/d/o8cg41BjZR9k_Q) -o model_FR_ArcFaceLoss_0.7500_epoch_50_B3.weights

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100 87.8M  100 87.8M    0     0  7883k      0  0:00:11  0:00:11 --:--:-- 13.2M


In [15]:
# Импорт кода модели распознавания
from recognizer import FaceRecorgnizer, ArcFaceLoss

/content/recognizer.py:68: SyntaxWarning: invalid escape sequence '\s'
  arcface_loss =-\sum^{m}_{i=1}log


In [16]:
# подгружаем модель и веса в переменную
fr_arc_model = FaceRecorgnizer(model_name="efficientnet_b3", loss_class=ArcFaceLoss, feat_dim=512).to(DEVICE)
fr_arc_model.load_state_dict(torch.load("model_FR_ArcFaceLoss_0.7500_epoch_50_B3.weights", weights_only=True, map_location=torch.device('cpu')))
fr_arc_model.eval()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/49.3M [00:00<?, ?B/s]

FaceRecorgnizer(
  (backbone): Backbone(
    (backbone): EfficientNetFeatures(
      (conv_stem): Conv2d(3, 40, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn1): BatchNormAct2d(
        40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
        (drop): Identity()
        (act): SiLU(inplace=True)
      )
      (blocks): Sequential(
        (0): Sequential(
          (0): DepthwiseSeparableConv(
            (conv_dw): Conv2d(40, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=40, bias=False)
            (bn1): BatchNormAct2d(
              40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
              (drop): Identity()
              (act): SiLU(inplace=True)
            )
            (aa): Identity()
            (se): SqueezeExcite(
              (conv_reduce): Conv2d(40, 10, kernel_size=(1, 1), stride=(1, 1))
              (act1): SiLU(inplace=True)
              (conv_expand): Conv2d(10, 40, kernel_size=(1, 1),

## Подготовка данных для функций расчёта IRM

In [66]:
# Формирование списка имен файлов для части 'query' и 'distractor'
query_img_names = glob.glob('./test_dataset_aligned/query/*')
distractors_img_names = glob.glob('./test_dataset_aligned/distractor/*')

In [75]:
# Формируем словарь из id лиц и относящихся к нему image_id
query_dict = {
    face_id: [
        f"./test_dataset_aligned/{row['set_type']}/{row['image_id']}"
        for _, row in group.iterrows()
    ]
    for face_id, group in df_test.groupby('id')
}

In [87]:
# Посмотрим правильно сформировался словарь
query_dict

{7: ['./test_dataset_aligned/distractor/140464.jpg'],
 49: ['./test_dataset_aligned/distractor/124062.jpg'],
 71: ['./test_dataset_aligned/distractor/075533.jpg'],
 87: ['./test_dataset_aligned/distractor/094385.jpg'],
 104: ['./test_dataset_aligned/distractor/173840.jpg'],
 135: ['./test_dataset_aligned/distractor/152387.jpg'],
 143: ['./test_dataset_aligned/distractor/063642.jpg'],
 154: ['./test_dataset_aligned/distractor/125640.jpg'],
 155: ['./test_dataset_aligned/distractor/094917.jpg'],
 236: ['./test_dataset_aligned/query/089250.jpg',
  './test_dataset_aligned/query/059473.jpg',
  './test_dataset_aligned/query/136054.jpg'],
 246: ['./test_dataset_aligned/distractor/030046.jpg'],
 251: ['./test_dataset_aligned/query/123543.jpg',
  './test_dataset_aligned/query/025408.jpg',
  './test_dataset_aligned/query/138929.jpg'],
 272: ['./test_dataset_aligned/distractor/074812.jpg'],
 293: ['./test_dataset_aligned/distractor/029647.jpg'],
 307: ['./test_dataset_aligned/distractor/038085.jp

In [77]:
# Получаем эмбеддинги для query и distractor
query_embeddings = compute_embeddings(fr_arc_model, query_img_names)
distractors_embeddings = compute_embeddings(fr_arc_model, distractors_img_names)

In [78]:
# Получаем косинусные сходства позитивных и негативных пар query и пар quert-distractor
cosine_query_pos = compute_cosine_query_pos(query_dict, query_img_names, query_embeddings)
cosine_query_neg = compute_cosine_query_neg(query_dict, query_img_names, query_embeddings)
cosine_query_distractors = compute_cosine_query_distractors(query_embeddings, distractors_embeddings)

In [88]:
# Считаем
image_thr = []
image_tpr = []
fprs = [0.5, 0.2, 0.1, 0.05]
for fpr in fprs:
    x, y = compute_ir(cosine_query_pos, cosine_query_neg,
                      cosine_query_distractors, fpr=fpr)
    image_thr.append(x)
    image_tpr.append(y)

In [90]:
for fpr, thr, tpr in zip(fprs, image_thr, image_tpr):
    print(f"TPR@FPR_{fpr} = {tpr:.4f}, threshold = {thr:.4f}")

TPR@FPR_0.5 = 0.9333, threshold = 0.0046
TPR@FPR_0.2 = 0.7867, threshold = 0.0945
TPR@FPR_0.1 = 0.7200, threshold = 0.1435
TPR@FPR_0.05 = 0.6667, threshold = 0.1875


## Выводы:
Разработанная модель распознавания лиц показывает перспективные результаты, подтверждающие правильность выбранного подхода и реализацию базового пайплайна. Модель может быть использована в приложениях, где допустим умеренный уровень ложных допущений (FPR ~0.1-0.2). Основным направлением для дальнейшего развития является повышение надежности модели в строгих условиях (увеличение TPR@FPR=0.05). Модель успешно демонстрирует понимание принципов верификации лиц и методов оценки подобных систем.